In [13]:
import os

In [14]:
%pwd

'd:\\Text-Summerization'

In [15]:
os.chdir('../')

In [16]:
%pwd

'd:\\'

In [17]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [18]:
from Textsummerizer.constant import *
from Textsummerizer.utils.common import read_yaml,create_directories

class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config=self.config.data_transformation

        create_directories([config.root_dir])
        data_transformation_config = DataTransformationConfig(
             root_dir=config.root_dir,
             data_path=config.data_path,
             tokenizer_name=config.tokenizer_name
                            
            )
        return data_transformation_config
    
    






In [19]:
import os
from Textsummerizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

In [20]:
class DataTransformation:
    def __init__(
            self,
            config: DataTransformationConfig):
        self.config = config,
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)



    def convert_examples_to_featurs(self,example_batch):
        input_encodings=self.tokenizer(example_batch['dialogue'],max_Length=1024,truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings= self.tokenizer(example_batch['summary'],max_Length=128,truncation=True)

        return {
            'input_ids':input_encodings['input_ids'],
            'attention_mask':input_encodings['attention_mask'],
            'labels':target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

        

In [21]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config\\config.yaml'